In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic

In [2]:
data = pd.read_csv('balanced_data.csv')

In [3]:
tops = data[data['category'] == 'Shirts/Tops']
reviews = tops['text'].tolist()
ratings = tops['rating'].astype(int).tolist()


In [4]:
reviews = [str(i) for i in reviews]

In [5]:
tops.columns

Index(['Unnamed: 0', 'rating', 'parent_asin', 'text', 'title',
       'average_rating', 'category'],
      dtype='object')

In [6]:
#topic_model = BERTopic(language="english", verbose=True)
#topics, probs = topic_model.fit_transform(reviews)

In [7]:
#topic_model.get_topic_info()

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

In [9]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
topic_model = BERTopic(embedding_model=embedding_model)

In [10]:
vectorizer_model = CountVectorizer(
    stop_words="english",
    ngram_range=(1, 2)   
)

In [11]:

topic_model = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model)

In [12]:
topics, probs = topic_model.fit_transform(reviews)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

In [ ]:
tops['topic'] = topics
tops['topic_probs'] = probs

In [ ]:
tops_cleaned = tops[tops['topic'] != -1]

# Calculate sentiment polarity for each review
from textblob import TextBlob
import nltk

# Download required NLTK data (only needed first time)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')

# Polarity ranges from -1 (negative) to 1 (positive)
sentiment_scores = []
for review in tops_cleaned['text']:
    blob = TextBlob(str(review))
    sentiment_scores.append(blob.sentiment.polarity)

tops_cleaned['sentiment_polarity'] = sentiment_scores

print(f"Sentiment scores calculated. Range: {min(sentiment_scores):.3f} to {max(sentiment_scores):.3f}")
print(f"Mean sentiment: {np.mean(sentiment_scores):.3f}")

Sentiment scores calculated. Range: -1.000 to 1.000
Mean sentiment: 0.156


In [15]:
tops_cleaned.head()

,Unnamed: 0,rating,parent_asin,text,title,average_rating,category,topic,topic_probs,sentiment_polarity
0,0,2.0,B08NHJ5S5H,Not long enough. I like shirt that cover or be...,Dokotoo Women's Ladies Spring Basic Ribbed Str...,3.6,Shirts/Tops,11,0.987920,0.012500
3,3,3.0,B071F91SCM,"Very pretty, but the post is super thick and t...",925 Sterling Silver Cubic Zirconia Purple Butt...,4.7,Shirts/Tops,17,0.234251,0.119444
9,9,4.0,B08JGGTCB9,I got it for my bf for his birthday was disapp...,Jordan Paris Saint-Germain Long-Sleeve T-Shirt...,4.0,Shirts/Tops,323,1.000000,-0.125000
17,17,3.0,B01IP4GEA2,Small and itchy.,Romwe Women's Striped Crewneck Short Sleeve Lo...,3.3,Shirts/Tops,160,0.963756,-0.250000
18,18,5.0,B08B5QZYWD,I love this shirt. It’s soft on my skin and d...,Happy Sailed Womens High Low Swing Pullover Pr...,3.1,Shirts/Tops,16,0.589095,0.292857


In [16]:
#topic_model.get_topic_info(1)['Representative_Docs']

print(topic_model.get_topic_info(2)['Representative_Docs'])

0    [[[VIDEOID:9bc533c84abc87e9f505bb102bba2cf8]] ...
Name: Representative_Docs, dtype: object


In [17]:
topic_model.get_representative_docs(45)

["These were a cute style, and the material was a pretty print. However, the coverage was a little too full in the back for me. It hit me low on the hip in a weird way.<br /><br />I think the rise was right at or just above my belly button, but it's been a few weeks since I returned these, so I can't remember exactly. I remember the waistband was very comfortable, almost loose, so there was no muffin top or anything like that.<br /><br />I also think I might have needed to size down a size or two. I considered re-ordering, but I wasn't sure about the cut of the back. I still sometimes think about trying to re-order in a smaller size to see if they would look different or better.<br /><br />The lack of a true tummy control lining was a bit of a disappointment. I also don't remember if these were fully lined, front and back. If they were not, that was another factor in the decision to return.<br /><br />However, these could be very cute, and a great buy, for someone who wants this fuller

In [18]:
topic_model.get_topic_info(45)

,Topic,Count,Name,Representation,Representative_Docs
0,45,85,45_pants_legs_pant_leg,"[pants, legs, pant, leg, sweatpants, panties, ...","[These were a cute style, and the material was..."


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,14838,-1_shirt_size_material_like,"[shirt, size, material, like, fit, small, br, ...",[When I normally buy something online I ALWAYS...
1,0,1726,0_dress_love dress_dresses_cute dress,"[dress, love dress, dresses, cute dress, pocke...","[Cute dress but it runs very small, Love this ..."
2,1,1023,1_tops_cute_love_beautiful,"[tops, cute, love, beautiful, fits, really, ni...","[cute top, Cute top, It is a very cute top]"
3,2,414,2_la_muy_es_el,"[la, muy, es, el, que, calidad, pero, tela, en...",[[[VIDEOID:9bc533c84abc87e9f505bb102bba2cf8]] ...
4,3,370,3_blouse_nice blouse_blouses_beautiful blouse,"[blouse, nice blouse, blouses, beautiful blous...","[Really like this blouse., cute blouse., Very ..."
...,...,...,...,...,...
380,379,10,379_order based_gathers_raise arms_chart ordered,"[order based, gathers, raise arms, chart order...",[I have a 36 chest size so I ordered an XL. It...
381,380,10,380_away gave_gave away_away_gave,"[away gave, gave away, away, gave, didn givwe,...","[I gave it away, gave it away, I gave it away]"
382,381,10,381_glue_wow broke_cute playful_like play,"[glue, wow broke, cute playful, like play, han...",[Its cute but made with such cheap metal. Feel...
383,382,10,382_wonder 10_clearly slightly_awkward fan_sea...,"[wonder 10, clearly slightly, awkward fan, sea...",[very thin material and runs a little small. I...


In [25]:
# Get topic info and remove topic = -1 (outliers)
topic_info = topic_model.get_topic_info()
print("Topic info before removing outliers:")
print(topic_info)
print(f"\nTotal topics including outliers: {len(topic_info)}")

# Remove topic = -1 from topic info
topic_info_cleaned = topic_info[topic_info['Topic'] != -1].copy()
print(f"\nTopic info after removing outliers (topic = -1):")
print(topic_info_cleaned)
print(f"\nTotal topics after removing outliers: {len(topic_info_cleaned)}")

Topic info before removing outliers:
     Topic  Count                                               Name  \
0       -1  14838                        -1_shirt_size_material_like   
1        0   1726              0_dress_love dress_dresses_cute dress   
2        1   1023                         1_tops_cute_love_beautiful   
3        2    414                                     2_la_muy_es_el   
4        3    370      3_blouse_nice blouse_blouses_beautiful blouse   
..     ...    ...                                                ...   
380    379     10   379_order based_gathers_raise arms_chart ordered   
381    380     10                  380_away gave_gave away_away_gave   
382    381     10          381_glue_wow broke_cute playful_like play   
383    382     10  382_wonder 10_clearly slightly_awkward fan_sea...   
384    383     10                   383_member_pay return_sold_largw   

                                        Representation  \
0    [shirt, size, material, like, fit, 

In [28]:
# Display cleaned topic info (without topic = -1)
topic_info_cleaned


,Topic,Count,Name,Representation,Representative_Docs
1,0,1726,0_dress_love dress_dresses_cute dress,"[dress, love dress, dresses, cute dress, pocke...","[Cute dress but it runs very small, Love this ..."
2,1,1023,1_tops_cute_love_beautiful,"[tops, cute, love, beautiful, fits, really, ni...","[cute top, Cute top, It is a very cute top]"
3,2,414,2_la_muy_es_el,"[la, muy, es, el, que, calidad, pero, tela, en...",[[[VIDEOID:9bc533c84abc87e9f505bb102bba2cf8]] ...
4,3,370,3_blouse_nice blouse_blouses_beautiful blouse,"[blouse, nice blouse, blouses, beautiful blous...","[Really like this blouse., cute blouse., Very ..."
5,4,300,4_suit_bathing_swimsuit_bathing suit,"[suit, bathing, swimsuit, bathing suit, bottom...",[Cute bathing suit but material was a little t...
...,...,...,...,...,...
380,379,10,379_order based_gathers_raise arms_chart ordered,"[order based, gathers, raise arms, chart order...",[I have a 36 chest size so I ordered an XL. It...
381,380,10,380_away gave_gave away_away_gave,"[away gave, gave away, away, gave, didn givwe,...","[I gave it away, gave it away, I gave it away]"
382,381,10,381_glue_wow broke_cute playful_like play,"[glue, wow broke, cute playful, like play, han...",[Its cute but made with such cheap metal. Feel...
383,382,10,382_wonder 10_clearly slightly_awkward fan_sea...,"[wonder 10, clearly slightly, awkward fan, sea...",[very thin material and runs a little small. I...


#### Training the Model

##### Linear Regression

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Convert topic_probs Series to a 2D numpy array
topic_features = np.vstack(tops_cleaned['topic_probs'].values)

# Add sentiment polarity as an additional feature
sentiment_features = tops_cleaned['sentiment_polarity'].values.reshape(-1, 1)

# Combine topic probabilities with sentiment features
combined_features = np.hstack([topic_features, sentiment_features])

print(f"Feature shape: {combined_features.shape}")
print(f"  - Topic probabilities: {topic_features.shape[1]} features")
print(f"  - Sentiment polarity: 1 feature")

X_train, X_test, y_train, y_test = train_test_split(
    combined_features, tops_cleaned['rating'], test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)

print("MSE:", mse)

from sklearn.metrics import r2_score
r2_model = r2_score(y_test, preds)
print ("R-squared:", r2_model)

Feature shape: (19095, 2)
  - Topic probabilities: 1 features
  - Sentiment polarity: 1 feature
MSE: 1.353701263781489
R-squared: 0.3138095442810107


#### XGBoost

In [22]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6
)

xgb_model.fit(X_train, y_train)

# Make predictions on test set
xgb_preds = xgb_model.predict(X_test)

# Calculate evaluation metrics
xgb_mse = mean_squared_error(y_test, xgb_preds)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, xgb_preds)

print("XGBoost Model Performance:")
print(f"MSE: {xgb_mse:.4f}")
print(f"RMSE: {xgb_rmse:.4f}")
print(f"R² Score: {xgb_r2:.4f}")


XGBoost Model Performance:
MSE: 1.3325
RMSE: 1.1543
R² Score: 0.3246


#### Random Forest

In [33]:
#X = topic_info_cleaned.iloc[:, 2:] #topic probs and sentiment score
#y = topic_info_cleaned['rating']
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(combined_features, tops_cleaned['rating'], test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=42
)

rf.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)

print("MSE:", mse)

r2 = r2_score(y_test, preds)
print(f"R-squared: {r2}")

MSE: 1.353701263781489
R-squared: 0.3138095442810107
